# Embedding Libraries from docker images

This notebook holds an exploration of the docker images library embedding using swivel. It expects the result of swivel to be accessible in a `./swivel_data` folder, and the result of dockerhub_crawler to be accessible in a `./packages` folder.

In [1]:
import pandas as pd
em = pd.read_csv("./swivel_data/row_embedding.tsv", sep="\t", index_col=0, header=None)

In [2]:
em.dropna()
em.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32768 entries, inherits to librte-pmd-ring17.11
Columns: 300 entries, 1 to 300
dtypes: float64(300)
memory usage: 76.5+ MB


In [3]:
from MulticoreTSNE import MulticoreTSNE as TSNE
array = em.values
tsne = TSNE(n_jobs=8, n_components=2)
Y = tsne.fit_transform(array)

In [4]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [5]:
packages_list = !find packages -type f
def get_images_libraries():
    for path in packages_list:
        with open(path) as json_file:
            try:
                package = json.load(json_file)
                libs = []
                libs += map(lambda x: ['native'] + x, [natpack for natpack in package["packages"]["native"]])
                libs += map(lambda x: ['python3'] + x , [natpack for natpack in package["packages"]["python3"]])
                libs += map(lambda x: ['node'] + x , [natpack for natpack in package["packages"]["node"]])
                yield libs
            except Exception as e:
                print(e)
                pass

In [6]:
import json
from tqdm import tqdm
count = 0
image_libs = None
for image_libs_list in tqdm(get_images_libraries()):
    count += 1
    if count != 1:
        try:
            image_libs = pd.concat([image_libs, pd.DataFrame(image_libs_list).set_index(1)])
            image_libs = image_libs[~image_libs.index.duplicated(keep='first')]
        except Exception as e:
            pass
    else:
        image_libs = pd.DataFrame(image_libs_list).set_index(1)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [7]:
image_libs.dropna(inplace=True)
em.loc['adduser'].values.size

300

In [8]:
from sklearn import preprocessing
import numpy as np
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

colormap = {'native': 'lightblue', 'node': 'green', 'python3': 'blue'}
output_notebook()
min_max_scaler = preprocessing.MinMaxScaler()

size_norm = np.log(image_libs[3].values.reshape(-1, 1))*0.7 + 3
number = 70000
colors = [colormap[image_libs[0][x]] if x in image_libs.index else 'grey' for x in em.index]
Y_tin = pd.DataFrame(Y)
source = ColumnDataSource(data=dict(
    x=Y_tin[0][:number],
    y=Y_tin[1][:number],
    desc=em.index[:number],
    color=colors[:number],
    size=image_libs[3][:number],
    size_norm=size_norm[:number]
))

TOOLTIPS = [
    ("name", "@desc"),
    ("size", "@size"),
]

p = figure(plot_width=900, plot_height=600, tooltips=TOOLTIPS, 
           title="Mouse over the dots")

p.circle('x', 'y',  source=source, color='color', size='size_norm', fill_alpha=0.2, line_alpha=0.4)

show(p)

Loading BokehJS ...

/home/tristan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.


In [9]:
packages_list = !find packages -type f
packages_list[0]

'packages/bp/bpedroso/medium-cicd-buzz.json'

In [10]:
def get_images_vector():
    for path in packages_list:
        package_name = path[13:]
        with open(path) as json_file:
            try:
                package = json.load(json_file)
                libs = []
                libs += [em.loc[natpack[0]].values for natpack in package["packages"]["native"]]
                libs += [em.loc[natpack[0]].values for natpack in package["packages"]["python3"]]
                libs += [em.loc[natpack[0]].values for natpack in package["packages"]["node"]]
                image = {"name": package['image'], "distro": package['distribution'], "size": package["size"], "vector": np.array(libs).mean(axis=0)}
                yield image
            except Exception as e:
                #print(e)
                pass

In [11]:
from tqdm import tqdm
images_vectors = [image for image in tqdm(get_images_vector())]

10it [00:00, 13.19it/s]/home/tristan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice.
  # This is added back by InteractiveShellApp.init_path()
/home/tristan/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
56384it [54:18, 17.30it/s]


In [12]:
images_vectors = pd.DataFrame(images_vectors).dropna().set_index('name')

In [ ]:
images_vectors['distro'] = images_vectors['distro'].astype('category')
images_vectors['distro'].cat.categories

In [ ]:
images_vectors= pd.read_pickle('images_vectors.pickle')

In [ ]:
images_vectors.groupby('distro').count()

In [ ]:
images_vectors.head()

In [ ]:
images_vectors['distro'] = images_vectors['distro'].cat.remove_categories(["rhel", "ol", "amzn", "fedora", "centos"])

In [ ]:
images_vectors['distro'] = images_vectors['distro'].cat.add_categories(['rhel-based'])
images_vectors['distro'] = images_vectors['distro'].fillna('rhel-based')
images_vectors.groupby('distro').count()

In [ ]:
images_vectors = images_vectors[images_vectors.distro.isin(["alpine", "debian", "ubuntu", "rhel-based"])]
images_vectors['distro'] = images_vectors['distro'].cat.remove_categories(["", "linuxmint", 'opensuse-leap', 'photon', 'raspbian', 'red'])
images_vectors.groupby('distro').count()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
images_vectors['size'].hist(bins=40)
plt.show()

In [12]:
import numpy as np
X = np.stack(images_vectors['vector'].values)
X.shape
#X = normalize(X, axis=1, order=2)

(16138, 300)

In [14]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.utils import to_categorical

lencoder = LabelEncoder()
y = lencoder.fit_transform(images_vectors['distro'].values)
oencoder = OneHotEncoder()
y = oencoder.fit_transform(y.reshape(-1, 1)).todense()
y

/home/tristan/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


matrix([[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        ...,
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)

NameError: name 'y' is not defined

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model = Sequential([
    Dense(100, input_shape=(300,)),
    Activation('relu'),
    Dense(100),
    Activation('relu'),
    Dense(4, activation='softmax'),
])
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=3)

history = model.fit(X_train, y_train, validation_split=0.2, epochs=30)

In [ ]:
y_pred = model.predict(X_test)
y_pred = (y_pred == y_pred.max(axis=1)[:,None]).astype(float)
from sklearn.metrics import confusion_matrix
c_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print(c_matrix)
plt.imshow(c_matrix)
print(lencoder.classes_)

In [1]:
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmaxscaler = MinMaxScaler()
y_reg = images_vectors['size'].values.reshape(-1, 1)
max_size = y_reg.max()
y_reg = y_reg/max_size

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_reg, test_size=0.2, random_state=2)

In [ ]:
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
regmodel = Sequential([
    Dense(100, input_shape=(300,)),
    Activation('relu'),
    Dense(100),
    Activation('relu'),
    Dense(1,  activation="linear"),
])
regmodel.compile(optimizer='adam', loss='mean_squared_error', metrics=["mae"])

In [ ]:
history = regmodel.fit(X_train, y_train, validation_split=0.2, epochs=30)

In [ ]:
print(history.history['val_mean_absolute_error'][-1]*max_size)